In [7]:
import pandas as pd
import numpy as np
import os
import logging
import time
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy.utils import complex_json_handler, unpack_data
from yfpy import get_logger
from dotenv import load_dotenv
from gspread_pandas import Spread

from utils import (
    get_season,
    league_season_info,
    google_sheet_upload,
    sql_upload_table,
    sql_grab_table,
)


load_dotenv()
pd.set_option("display.max_colwidth", 999)
pd.set_option("display.max_columns", 999)
pd.set_option('display.max_rows', 100)

LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = Path.cwd().parents[0]
# DATA_DIR = PATH / "data"

# SEASON = get_season()
# NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()

# TODAY = np.datetime64("today", "D")
# TODAY = np.datetime64("2021-09-28")
# NFL_WEEK = NFL_DATES_DF["Week"][(NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)].values[0]
# LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
LEAGUE_ID = "777818"
# GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
GAME_ID = "273"
# WEEKS = list(range(NFL_WEEK, 0, -1))

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

try:
    yahoo_query = YahooFantasySportsQuery(
        auth_dir=PATH,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )

except Exception as e:
    print(e)

In [8]:
# players = sql_grab_table("Players")
players = pd.read_csv('players.csv')
player_keys = list(players["player_key"])
player_stats = pd.DataFrame()
for k in player_keys:
    for w in range(1, 18):
        try:
            r1 = yahoo_query.get_player_stats_by_week(k, w)
            r2 = yahoo_query.get_player_percent_owned_by_week(k, w)

        except Exception as e:
            if 'token_expired' in str(e):
                print(e)
                yahoo_query._authenticate()
            else:
                print(f'Error with player stats at player_key {k} and week {w}.\n{e}\nRetrying in 15 minutes.')
                time.sleep(1800)
                yahoo_query._authenticate()

            r1 = yahoo_query.get_player_stats_by_week(k, w)
            r2 = yahoo_query.get_player_percent_owned_by_week(k, w)

        time.sleep(5)
        data = complex_json_handler(r1)
        player = pd.json_normalize(data)
        stats = data["player_stats"]["stats"]
        player_stat_week = pd.DataFrame()

        for r in stats:
            stat = pd.json_normalize(complex_json_handler(r["stat"]))
            stat["player_key"] = player["player_key"]
            stat["week"] = player["player_stats.week"]
            stat["total_points_week"] = player["player_points.total"]
            player_stat_week = pd.concat([player_stat_week, stat])

        ownership = pd.json_normalize(complex_json_handler(r2))
        if 'percent_owned.value' not in ownership.columns:
            ownership['percent_owned.value'] = 0

        ownership = ownership[["player_key", "percent_owned.value", "percent_owned.delta"]]

        player_stat_week = player_stat_week.merge(
            ownership,
            how="outer",
            left_on="player_key",
            right_on="player_key",
        )

        player_stats = pd.concat([player_stats, player_stat_week])

2022-06-18 22:33:03.521 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


2022-06-18 23:33:05.530 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


2022-06-19 00:33:10.551 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


...

Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


2022-06-19 16:34:00.222 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


2022-06-19 17:34:00.913 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/273/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


In [ ]:
class league_season_data(object):

    def __init__(
        self,
        auth_dir: Path,
        league_id: str,
        game_id: int = None,
        game_code: str = "nfl",
        offline: bool = False,
        all_output_as_json: bool = False,
        consumer_key: str = None,
        consumer_secret: str = None,
        browser_callback: bool = True,
    ):

        self._auth_dir = auth_dir
        self._consumer_key = consumer_key
        self._consumer_secret = consumer_secret
        self._browser_callback = browser_callback

        self.league_id = league_id
        self.game_id = game_id
        self.game_code = game_code

        self.offline = offline
        self.all_output_as_json = all_output_as_json

    def _yahoo_query(self):

        try:
            yahoo_query = YahooFantasySportsQuery(
                auth_dir=self._auth_dir,
                league_id=self.league_id,
                game_id=self.game_id,
                game_code=self.game_code,
                offline=self.offline,
                all_output_as_json=self.all_output_as_json,
                consumer_key=self._consumer_key,
                consumer_secret=self._consumer_secret,
                browser_callback=self._browser_callback,
            )

            return yahoo_query

        except Exception as e:
            print(e)


    def metadata(self, first_time="no"):
        response = complex_json_handler(yahoo_query.get_league_metadata())
        league_metadata = pd.json_normalize(response)

        league_metadata["game_id"] = self.game_id
        league_metadata.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=league_metadata,
                table_name="LeagueMetaData",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=league_metadata,
                table_name="LeagueMetaData",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return league_metadata


    def set_roster_pos_stat_cat(self, first_time="no"):
        response = complex_json_handler(yahoo_query.get_league_settings())

        league_settings = pd.json_normalize(response)
        league_settings.drop(
            ["roster_positions", "stat_categories.stats", "stat_modifiers.stats"],
            axis=1,
            inplace=True,
        )

        league_settings["game_id"] = self.game_id
        league_settings["league_id"] = self.league_id
        league_settings.drop_duplicates(ignore_index=True, inplace=True)

        roster_positions = pd.DataFrame()
        for r in response["roster_positions"]:
            row = pd.json_normalize(complex_json_handler(r["roster_position"]))
            roster_positions = pd.concat([roster_positions, row])

        roster_positions["game_id"] = self.game_id
        roster_positions["league_id"] = self.league_id
        roster_positions.drop_duplicates(ignore_index=True, inplace=True)

        stat_categories = pd.DataFrame()
        for r in response["stat_categories"]["stats"]:
            row = pd.json_normalize(complex_json_handler(r["stat"]))
            try:
                row["position_type"] = complex_json_handler(
                    complex_json_handler(r["stat"])["stat_position_types"][
                        "stat_position_type"
                    ]
                )["position_type"]
            except:
                pass

            try:
                row["is_only_display_stat"] = complex_json_handler(
                    complex_json_handler(r["stat"])["stat_position_types"][
                        "stat_position_type"
                    ]
                )["is_only_display_stat"]
            except:
                row["is_only_display_stat"] = 0

            try:
                row.drop("stat_position_types.stat_position_type", axis=1, inplace=True)
            except:
                pass

            stat_categories = pd.concat([stat_categories, row])

        stat_categories["game_id"] = self.game_id
        stat_categories["league_id"] = self.league_id

        stat_modifiers = pd.DataFrame()
        for r in response["stat_modifiers"]["stats"]:
            row = pd.json_normalize(complex_json_handler(r["stat"]))
            stat_modifiers = pd.concat([stat_modifiers, row])

        stat_modifiers.rename(columns={"value": "stat_modifier"}, inplace=True)

        stat_categories = stat_categories.merge(
            stat_modifiers, how="outer", on="stat_id"
        )
        stat_categories.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=league_settings,
                table_name="LeagueSettings",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

            sql_upload_table(
                dataframe=roster_positions,
                table_name="RosterPositions",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

            sql_upload_table(
                dataframe=stat_categories,
                table_name="StatCategories",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=league_settings,
                table_name="LeagueSettings",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

            sql_upload_table(
                dataframe=roster_positions,
                table_name="RosterPositions",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

            sql_upload_table(
                dataframe=stat_categories,
                table_name="StatCategories",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return league_settings, roster_positions, stat_categories


    def teams_list(self, first_time="no"):
        response = yahoo_query.get_league_teams()
        teams = pd.DataFrame()
        for r in response:
            team = pd.json_normalize(complex_json_handler(r["team"]))
            try:
                manager = pd.json_normalize(
                    complex_json_handler(team["managers.manager"][0])
                )
            except:
                manager = pd.json_normalize(
                    complex_json_handler(team["managers"][0][0]["manager"])
                )
            team = pd.concat([team, manager], axis=1)
            if "clinched_playoffs" not in team.columns:
                team["clinched_playoffs"] = 0
            team = team[
                [
                    "clinched_playoffs",
                    "has_draft_grade",
                    "league_scoring_type",
                    "name",
                    "number_of_moves",
                    "number_of_trades",
                    "team_id",
                    "team_key",
                    "waiver_priority",
                    "roster_adds.coverage_type",
                    "roster_adds.coverage_value",
                    "roster_adds.value",
                    "guid",
                    "manager_id",
                    "nickname",
                ]
            ]

            teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)

        teams["name"] = teams["name"].str.decode("utf-8")

        teams["game_id"] = self.game_id
        teams["league_id"] = self.league_id
        teams.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=teams,
                table_name="Teams",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=teams,
                table_name="Teams",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return teams


    def players_list(self, first_time="no"):
        response = yahoo_query.get_league_players()
        players = pd.DataFrame()
        for r in response:
            player = pd.json_normalize(complex_json_handler(r["player"]))
            try:
                draft_analysis = pd.json_normalize(complex_json_handler(yahoo_query.get_player_draft_analysis(player["player_key"][0])))

            except Exception as e:
                if 'token_expired' in str(e):
                    yahoo_query._authenticate()
                else:
                    print('Retrying after sleeping 15 min.')
                    time.sleep(900)
                    yahoo_query._authenticate()

                draft_analysis = pd.json_normalize(complex_json_handler(yahoo_query.get_player_draft_analysis(player["player_key"][0])))

            draft_analysis = draft_analysis[
                [
                    "draft_analysis.average_pick",
                    "draft_analysis.average_round",
                    "draft_analysis.average_cost",
                    "draft_analysis.percent_drafted",
                ]
            ]
            player = pd.concat([player, draft_analysis], axis=1)
            if "status" not in player.columns:
                player["status"] = np.nan

            players = pd.concat([players, player], ignore_index=True)
            time.sleep(2.5)


        players["game_id"] = self.game_id
        players["league_id"] = self.league_id
        players.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=players,
                table_name="Players",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=players,
                table_name="Players",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return players


    def draft_results(self, first_time="no"):
        response = yahoo_query.get_league_draft_results()
        draft_results = pd.DataFrame()
        for r in response:
            row = pd.json_normalize(complex_json_handler(r["draft_result"]))
            draft_results = pd.concat([draft_results, row])

        draft_results["game_id"] = self.game_id
        draft_results["league_id"] = self.league_id
        draft_results.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=draft_results,
                table_name="DraftResults",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=draft_results,
                table_name="DraftResults",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return draft_results


    def matchups_by_week(self, first_time="no"):
        m = []
        team_a = pd.DataFrame()
        team_b = pd.DataFrame()
        response = yahoo_query.get_league_matchups_by_week(1)
        for data in response:
            m.append(complex_json_handler(data["matchup"]))
        matchups = pd.DataFrame()
        for r in m:
            matchup = pd.json_normalize(r)
            matchup = matchup[
                [
                    "is_consolation",
                    "is_matchup_recap_available",
                    "is_playoffs",
                    "is_tied",
                    "matchup_recap_title",
                    "matchup_recap_url",
                    "status",
                    "week",
                    "week_end",
                    "week_start",
                    "winner_team_key",
                ]
            ]
            try:
                team_a = pd.json_normalize(
                    complex_json_handler(r["matchup_grades"][0]["matchup_grade"])
                )
                team_a["points"] = complex_json_handler(r["teams"][0]["team"])[
                    "team_points"
                ]["total"]
                team_a["projected_points"] = complex_json_handler(
                    r["teams"][0]["team"]
                )["team_projected_points"]["total"]

            except:
                team_a = pd.json_normalize(complex_json_handler(r["teams"][0]["team"]))
                team_a = team_a[
                    ["team_key", "team_points.total", "team_projected_points.total"]
                ]
                team_a["grade"] = np.nan

            team_a = team_a.add_prefix("team_a_")

            try:
                team_b = pd.json_normalize(
                    complex_json_handler(r["matchup_grades"][1]["matchup_grade"])
                )
                team_b["points"] = complex_json_handler(r["teams"][1]["team"])[
                    "team_points"
                ]["total"]
                team_b["projected_points"] = complex_json_handler(
                    r["teams"][1]["team"]
                )["team_projected_points"]["total"]

            except:
                team_b = pd.json_normalize(complex_json_handler(r["teams"][1]["team"]))
                team_b = team_b[
                    ["team_key", "team_points.total", "team_projected_points.total"]
                ]
                team_b["grade"] = np.nan
            team_b = team_b.add_prefix("team_b_")

            matchup = pd.concat([matchup, team_a, team_b], axis=1)

            matchups = pd.concat([matchups, matchup])

        try:
            matchups.drop(["teams", "matchup_grades"], axis=1, inplace=True)

        except:
            pass

        matchups["game_id"] = self.game_id
        matchups["league_id"] = self.league_id
        matchups.drop_duplicates(inplace=True, ignore_index=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=matchups,
                table_name="Matchups",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=matchups,
                table_name="Matchups",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return matchups


    def team_standings(self, first_time="no"):
        standings = pd.DataFrame()
        i = 1
        while True:
            try:
                response = yahoo_query.get_team_standings(i)
                if bool(response) == True:
                    row = pd.json_normalize(complex_json_handler(response))
                    row["team_id"] = i
                    standings = pd.concat([standings, row])
                    i += 1
                else:
                    break
            except:
                break

        standings["game_id"] = self.game_id
        standings["league_id"] = self.league_id
        standings.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=standings,
                table_name="Standings",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=standings,
                table_name="Standings",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return standings


    def team_roster_by_week(self, first_time="no"):
        team_rosters = pd.DataFrame()
        for t in range(1, 13):
            try:
                for w in range(1, 19):
                    try:
                        response = complex_json_handler(yahoo_query.get_team_roster_by_week(t, w))
                        row = pd.json_normalize(complex_json_handler(response["players"][0]["player"]))
                        row["team_id"] = t
                        row["week"] = w
                        team_rosters = pd.concat([team_rosters, row])
                        time.sleep(1)
                    except:
                        print(f'team: {t}\nweek: {w}')
                        continue
            except:
                print(f'team: {t}\nweek: {w}')
                continue

        team_rosters["game_id"] = self.game_id
        team_rosters["league_id"] = self.league_id
        team_rosters.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=team_rosters,
                table_name="TeamRosters",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=team_rosters,
                table_name="TeamRosters",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return team_rosters


    def player_stats_by_week(self, first_time="no"):
        players = sql_grab_table("Players")
        player_keys = list(players["player_key"])
        player_stats = pd.DataFrame()
        for k in player_keys:
            for w in range(1, 18):
                try:
                    r1 = yahoo_query.get_player_stats_by_week(k, w)
                    r2 = yahoo_query.get_player_percent_owned_by_week(k, w)

                except Exception as e:
                    if 'token_expired' in str(e):
                        print(e)
                        yahoo_query._authenticate()
                    else:
                        print(f'Error with player stats at player_key {k} and week {w}.\n{e}\nRetrying in 15 minutes.')
                        time.sleep(1800)
                        yahoo_query._authenticate()

                    r1 = yahoo_query.get_player_stats_by_week(k, w)
                    r2 = yahoo_query.get_player_percent_owned_by_week(k, w)

                time.sleep(4.8)
                data = complex_json_handler(r1)
                player = pd.json_normalize(data)
                stats = data["player_stats"]["stats"]
                player_stat_week = pd.DataFrame()

                for r in stats:
                    stat = pd.json_normalize(complex_json_handler(r["stat"]))
                    stat["player_key"] = player["player_key"]
                    stat["week"] = player["player_stats.week"]
                    stat["total_points_week"] = player["player_points.total"]
                    player_stat_week = pd.concat([player_stat_week, stat])

                ownership = pd.json_normalize(complex_json_handler(r2))
                if 'percent_owned.value' not in ownership.columns:
                    ownership['percent_owned.value'] = 0

                ownership = ownership[["player_key", "percent_owned.value", "percent_owned.delta"]]

                player_stat_week = player_stat_week.merge(
                    ownership,
                    how="outer",
                    left_on="player_key",
                    right_on="player_key",
                )

        player_stats = pd.concat([player_stats, player_stat_week])

        player_stats["game_id"] = self.game_id
        player_stats["league_id"] = self.leauge_id
        player_stats.drop_duplicates(ignore_index=True, inplace=True)

        if str(first_time).upper() == "YES":
            sql_upload_table(
                dataframe=player_stats,
                table_name="PlayerStats",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == "NO":
            sql_upload_table(
                dataframe=player_stats,
                table_name="PlayerStats",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return player_stats


    def all_game_keys(self):
        response = unpack_data(yahoo_query.get_all_yahoo_fantasy_game_keys())
        league_keys = pd.read_csv("../assests/ID.csv")
        game_keys = pd.DataFrame()
        for r in response:
            row = pd.DataFrame(complex_json_handler(r["game"]), index=[0])
            game_keys = pd.concat([game_keys, row])

        game_keys.reset_index(drop=True, inplace=True)
        game_keys = game_keys[game_keys["season"] >= 2012]
        game_keys = game_keys.merge(
            league_keys,
            how="outer",
            left_on=["game_id", "season"],
            right_on=["game_id", "season"],
        )
        game_keys.drop_duplicates(ignore_index=True, inplace=True)

        sql_upload_table(
            dataframe=game_keys,
            table_name="GameKeys",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )

        return game_keys


    def all_nfl_weeks(self):
        game_keys = sql_grab_table("GameKeys")
        game_id = list(game_keys["game_id"])
        weeks = pd.DataFrame()
        for g in game_id:
            response = yahoo_query.get_game_weeks_by_game_id(g)
            for r in response:
                row = pd.json_normalize(complex_json_handler(r["game_week"]))
                row["game_id"] = g
                weeks = pd.concat([weeks, row])

        weeks.rename(columns={"display_name": "week"}, inplace=True)
        weeks.drop_duplicates(ignore_index=True, inplace=True)

        sql_upload_table(
            dataframe=weeks,
            table_name="NFLWeeks",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )

        return weeks

In [ ]:
SEASON = get_season()
NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()
# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
NFL_WEEK = NFL_DATES_DF["Week"][(NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)].values[0]
WEEKS = list(range(NFL_WEEK, 0, -1))



PATH = Path.cwd().parents[0]
LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

In [ ]:
first_time = "yes"
league = league_season_data(
    auth_dir=PATH,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)